In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils import shuffle

In [7]:
#Fungsi untuk menghitung Entropy
def entropy(y):
    #Mehitung jumlah kemunculan kelas, hasil berupa array [a, b]
    kemunculan_kelas = np.bincount(y)
    #Menghitung probabilitas kemunculan kelas, hasil berupa array [a, b]
    prob_kemunculan = kemunculan_kelas/len(y)
    #Return rumus entropy, dengan kondisi px tidak kurang dari 0
    return -np.sum([px * np.log2(px) for px in prob_kemunculan if px > 0])

#Kelas pembantu untuk menyimpan informasi node
class Node:
    #Parameter dari Node yang ada pada sebuah Decision Tree yang nilainya perlu disimpan
    #* digunakan untuk keyword only parameter
    def __init__(self, fitur=None, threshold=None, node_kiri=None, node_kanan=None, *, nilai_leaf=None):
        self.fitur = fitur
        self.threshold = threshold
        self.node_kiri = node_kiri
        self.node_kanan = node_kanan
        self.nilai_leaf = nilai_leaf
    
    #Fungsi pembantu untuk mengindikasikan bawah proses spilitting sudah sampai di node leaf
    def leaf_node(self):
        return self.nilai_leaf is not None
    
#Kelas Decision Tree
class DecisionTree:
    #Constructor dengan parameter untuk proses splitting
    def __init__(self, min_samples_split=2, max_depth=100, jumlah_fitur=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.jumlah_fitur = jumlah_fitur
        self.root = None
    
    #Fungsi untuk melakukan training 
    def fit(self, X, y):
        #Jumlah fitur = jumlah kolom dari x yang dipilih
        #Dengan kondisi jika nanti jumlah_fitur tidak didefinisikan maka jumlah fitur yang dipilih adalah jumlah maksimal dari fitur yang dipilih
        #Dan jumlah fitur yang dipilih tidak akan lebih besar dari jumlah_fitur yang ada
        self.jumlah_fitur = X.shape[1] if not self.jumlah_fitur else min(self.jumlah_fitur, X.shape[1])
        #Memulai pembentukan decision tree dari root
        self.root = self.tree(X, y)
    
    #Fungsi untuk membentuk decision tree
    def tree(self, X, y, depth=0):
        #Mendapatkan nilai dari jumlah sample/baris dan nilai dari jumlah fitur/kolom yang ada pada data hasil pre-processing
        n_samples, n_fitur = X.shape
        #Mendapatkan nilai dari total kelas yang ada pada data hasil pre-processing
        n_kelas = len(np.unique(y))
        
        #Stopping criteria untuk proses splitting
        #Jika kondisi terpenuhi
        if (depth >= self.max_depth
                or n_kelas == 1
                or n_samples < self.min_samples_split):
            #Maka proses splitting sudah sampai di leaf , dan nilai pada leaf bisa didapatkan menggunakan fungsi nilai_kelas_terbanyak
            nilai_leaf_node = self.nilai_kelas_terbanyak(y)
            #Menyimpan nilai leaf pada leaf_node
            return Node(nilai_leaf=nilai_leaf_node)
        
       
        #Memilih secara random fitur yang ada pada n_fitur dengan jumlah yang ada pada jumlah_fitur tanpa pengembalian
        fitur_dipilih = np.random.choice(n_fitur, self.jumlah_fitur, replace=False)
        
        #Mencari fitur terbaik dan threshold terbaik dari fitur yang telah dipilih secara random
        fitur_terbaik, threshold_terbaik = self.feature_extraction(X, y, fitur_dipilih)
        
        #Membuat decision node berdasarkan rule (attribut dan threshold terbaik) yang telah dihitung melalui beberapa proses
        #Memilih semua sample dengan fitur dan threshold terbaik
        split_kiri, split_kanan = self.split(X[:, fitur_terbaik], threshold_terbaik)
        #Memilih x dan y yang ada pada sample kiri untuk dijadikan sebagai sub decision node ( sudah dengan fitur dan threshold terbaik)
        node_kiri = self.tree(X[split_kiri, :], y[split_kiri], depth+1)
        #Memilih x dan y yang ada pada sample kanan  untuk dijadikan sebagai sub decision node ( sudah dengan fitur dan threshold terbaik)
        node_kanan = self.tree(X[split_kanan, :], y[split_kanan], depth+1)
        #Menyimpan nilai node
        return Node(fitur_terbaik, threshold_terbaik, node_kiri, node_kanan)
        
    #Fungsi untuk mendapatkan fitur dan threshold terbaik
    def feature_extraction(self, X, y, fitur_dipilih):
        best_gain = -1
        split_fitur, split_threshold = None, None
        
        for fitur_index in fitur_dipilih:
            #Memilih kolom dari fitur yang dipilih
            kolom_X = X[:, fitur_index]
            #Mendapatkan nilai fitur yang dipilih/threshold secara unik
            thresholds = np.unique(kolom_X)
            for threshold in thresholds:
                #Melakukan perhitungan entropy dan information gain untuk mendapatkan best threshold pada fungsi information_gain
                gain = self.information_gain(y, kolom_X, threshold)
                
                #Kondisi untuk mendapatkan fitur dan threshold terbaik untuk dijadikan sebagai rule pada decision node
                if gain > best_gain:
                    best_gain = gain
                    split_fitur = fitur_index
                    split_threshold = threshold
                    
        return split_fitur, split_threshold
        
    #Fungsi untuk menghitung information gain
    def information_gain(self, y, kolom_X, split_threshold):
        #Menghitung parent entropy
        parent_entropy = entropy(y)
        
        #Melakukan splitting dengan fungsi split
        split_kiri, split_kanan = self.split(kolom_X, split_threshold)
        
        if len(split_kiri) == 0 or len(split_kanan) == 0:
            return 0
        
        #Menghitung child entropy
        total_kelas = len(y)
        total_kelas_kiri = len(split_kiri)
        total_kelas_kanan = len(split_kanan)
        entropy_split_kiri = entropy(y[split_kiri])
        entropy_split_kanan = entropy(y[split_kanan])
        child_entropy = (total_kelas_kiri / total_kelas) * entropy_split_kiri + (total_kelas_kanan / total_kelas) * entropy_split_kanan
        
        #Information Gain
        ig = parent_entropy - child_entropy
        return ig
        
    #Fungsi untuk splitting    
    def split(self, kolom_X, split_threshold):
        split_kiri = np.argwhere(kolom_X <= split_threshold).flatten()
        split_kanan = np.argwhere(kolom_X > split_threshold).flatten()
        return split_kiri, split_kanan
        
        
    #Fungsi untuk mendapatkan nilai kelas terbanyak pada leaf node
    def nilai_kelas_terbanyak(self, y):
        #Menghitung jumlah kemunculan kelas pada leaf node
        counter = Counter(y)
        #Mendapatkan nilai kemunculan kelas terbanyak
        kemunculan_kelas_terbanyak = counter.most_common(1)[0][0]
        #Menyimpan nilai tersebut untuk di pass
        return kemunculan_kelas_terbanyak
    
    #Fungsi untuk melakukan prediksi
    def predict(self, X):
        #Bergerak dari awal root sampai ke leaf node
        return np.array([self.pergerakan_tree(x, self.root) for x in X])
    
    def pergerakan_tree(self, x, node):
        if node.leaf_node():
            return node.nilai_leaf
        
        if x[node.fitur] <= node.threshold:
            return self.pergerakan_tree(x, node.node_kiri)
        return self.pergerakan_tree(x, node.node_kanan)

In [8]:
def bootstrapped_dataset(X, y):
    sampel_data = X.shape[0]
    #Memilih secara random dari sampel data dengan pengembalian untuk dijadikan sebagai subset data yang berfungsi untuk training decision tree
    bootstrap_data = np.random.choice(sampel_data, size=sampel_data, replace=True)
    return X[bootstrap_data], y[bootstrap_data]

def voting(y):
    counter = Counter(y)
    #Mendapatkan nilai kemunculan kelas terbanyak
    kemunculan_kelas_terbanyak = counter.most_common(1)[0][0]
    #Menyimpan nilai tersebut untuk di pass
    return kemunculan_kelas_terbanyak

class RandomForest:
    #
    def __init__(self, jumlah_tree, min_samples_split=2, max_depth=100, jumlah_fitur=None):
        self.jumlah_tree = jumlah_tree
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.jumlah_fitur = jumlah_fitur
        #Untuk menyimpan informasi dari setiap decision tree yang telah dibuat
        self.array_tree = []
        
    def fit(self, X, y):
        self.array_tree = []
        #Melakukan training terhadap setiap decision dari sejumlah n decision tree berdasarkan bootstrapped dataset
        for single_tree in range(self.jumlah_tree):
            tree = DecisionTree(min_samples_split=self.min_samples_split, max_depth=self.max_depth, jumlah_fitur=self.jumlah_fitur)
            X_subset, y_subset = bootstrapped_dataset(X, y)
            tree.fit(X_subset, y_subset)
            #Menambahkan decision tree hasil training ke dalam array tree
            self.array_tree.append(tree)
            
    def predict(self, X):
        #
        tree_prediksi = np.array([tree.predict(X) for tree in self.array_tree])
        #
        tree_prediksi = np.swapaxes(tree_prediksi, 0, 1)
        #Melakukan voting dari n decision tree yang telah dibangun
        prediksi_y = [voting(sekumpulan_tree) for sekumpulan_tree in tree_prediksi]
        return np.array(prediksi_y)

In [9]:
data1 = pd.read_excel('DataSaham.xlsx')
data = data1.dropna()

ArrayData = np.array(data)
x = ArrayData[:2000,5:15]
X = x.astype(np.float)
Y_1D = ArrayData[:2000,15:16]
y_new = Y_1D.ravel()
y = y_new.astype(np.int)

<ipython-input-9-86342296351f>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = x.astype(np.float)
<ipython-input-9-86342296351f>:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y_new.astype(np.int)


In [10]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [11]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = (i * k_fold_size // 2)
            stop = start + k_fold_size
            mid = int(0.80 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [12]:
btss = BlockingTimeSeriesSplit(n_splits=4)

i=1
array_y_test = []
array_y_pred = []

for train_index, test_index in btss.split(X, y):
    print("Fold ", i)
    print("TRAIN :", train_index, "TEST :", test_index)
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    array_y_pred.append(get_score(RandomForest(jumlah_tree=10, max_depth=10), X_train, X_test, y_train, y_test))
    array_y_test.append(y_test)
    i+=1

Fold  1
TRAIN : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24

In [20]:
rf_matrix = confusion_matrix(array_y_test[0], array_y_pred[0])
#display(rf_matrix)

true_negative = rf_matrix[0][0]
false_negative = rf_matrix[1][0]
true_positive = rf_matrix[1][1]
false_positive = rf_matrix[0][1]

accuracy = (true_negative + true_positive) / (true_negative + false_negative + true_positive + false_positive)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
#specifity = true_negative / (true_negative + false_positive)
F1_Score = 2 * (recall * precision) / (recall + precision)

print('Accuracy :',format(float(accuracy)))
print('Precision :',format(float(precision)))
print('Recall :',format(float(recall)))
#print('Specifity : ',format(float(specifity)))
print('F1 Score :',format(float(F1_Score)))

Accuracy : 0.61
Precision : 0.71875
Recall : 0.6865671641791045
F1 Score : 0.7022900763358778
